<a href="https://colab.research.google.com/github/cristinarainich/MLTSA22_KRainich/blob/main/HW2/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
import pandas as pd
import numpy as np
import pylab as pl
%pylab inline

import statsmodels.api as sm # for the statistical analysis to find out if the TS is stationary
import glob #to sort files
from pandas.plotting import scatter_matrix #will make useful correlatin plots
import json #reads and writes json files, see programmatic data access notebooks
import os #this runs system level commands, see programmatic data access notebooks

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pl.style.use("https://raw.githubusercontent.com/fedhere/MLTSA22_FBianco/master/fbb.mplstyle")

Data preparation

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
#kaggle data
!pip install kaggle

In [5]:
!pwd

/content


In [6]:
cd /content/gdrive/MyDrive/

/content/gdrive/MyDrive


In [7]:
!ls .kaggle

kaggle.json  STOCK


In [8]:
cd /content/gdrive/MyDrive/.kaggle

/content/gdrive/MyDrive/.kaggle


In [9]:
!chmod 600 kaggle.json

In [10]:
import json
import os
envs = json.load(open("kaggle.json", "r"))
os.environ["KAGGLE_USERNAME"] = envs['username']
os.environ["KAGGLE_KEY"] = envs['key']

In [11]:
os.environ

environ({'NO_GCE_CHECK': 'True', 'GCS_READ_CACHE_BLOCK_SIZE_MB': '16', 'CLOUDSDK_CONFIG': '/content/.config', 'CUDA_VERSION': '11.1.1', 'PATH': '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin', 'HOME': '/root', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64', 'LANG': 'en_US.UTF-8', 'SHELL': '/bin/bash', 'LIBRARY_PATH': '/usr/local/cuda/lib64/stubs', 'SHLVL': '0', 'GCE_METADATA_TIMEOUT': '0', 'NCCL_VERSION': '2.7.8', 'NVIDIA_VISIBLE_DEVICES': 'all', 'DEBIAN_FRONTEND': 'noninteractive', 'CUDNN_VERSION': '8.0.5.39', 'LAST_FORCED_REBUILD': '20220127', 'JPY_PARENT_PID': '56', 'PYTHONPATH': '/env/python', 'DATALAB_SETTINGS_OVERRIDES': '{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=\\"172.28.0.2\\""],"debugAdapterMultiplexerPath":"/usr/local/bin/dap_multiplexer","enableLsp":true}', 'ENV': '/root/.bashrc', 'GLIBCXX_FORCE_NEW

In [12]:
!kaggle datasets

usage: kaggle datasets [-h]
                       {list,files,download,create,version,init,metadata,status}
                       ...
kaggle datasets: error: too few arguments


In [13]:
!kaggle datasets list -s stock

ref                                                       title                                      size  lastUpdated          downloadCount  
--------------------------------------------------------  ----------------------------------------  -----  -------------------  -------------  
mattiuzc/stock-exchange-data                              Stock Exchange Data                         5MB  2021-06-07 20:51:55          13599  
borismarjanovic/price-volume-data-for-all-us-stocks-etfs  Huge Stock Market Dataset                 492MB  2017-11-16 14:53:29          84112  
dgawlik/nyse                                              New York Stock Exchange                    31MB  2017-02-22 10:18:25          63194  
jacksoncrow/stock-market-dataset                          Stock Market Dataset                      522MB  2020-04-02 10:55:57          11966  
hk7797/stock-market-india                                 Stock Market India                          1GB  2021-01-02 03:44:23          

In [19]:
cd /content/gdrive/MyDrive/MLTSA22/HW2/Data

/content/gdrive/MyDrive/MLTSA22/HW2/Data


In [22]:
!kaggle datasets download borismarjanovic/price-volume-data-for-all-us-stocks-etfs

 99% 488M/492M [00:04<00:00, 139MB/s]
100% 492M/492M [00:04<00:00, 128MB/s]


In [25]:
!ls /content/gdrive/MyDrive/MLTSA22/HW2/Data

price-volume-data-for-all-us-stocks-etfs.zip


In [26]:
!unzip price-volume-data-for-all-us-stocks-etfs.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: Stocks/ergf.us.txt      
  inflating: Stocks/erh.us.txt       
  inflating: Stocks/eri.us.txt       
  inflating: Stocks/eric.us.txt      
  inflating: Stocks/erie.us.txt      
  inflating: Stocks/erii.us.txt      
  inflating: Stocks/erj.us.txt       
  inflating: Stocks/erm.us.txt       
  inflating: Stocks/ern.us.txt       
  inflating: Stocks/eros.us.txt      
  inflating: Stocks/eryp.us.txt      
  inflating: Stocks/es.us.txt        
  inflating: Stocks/esba.us.txt      
  inflating: Stocks/esbk.us.txt      
  inflating: Stocks/esca.us.txt      
  inflating: Stocks/esdi.us.txt      
  inflating: Stocks/esdiw.us.txt     
  inflating: Stocks/ese.us.txt       
  inflating: Stocks/esea.us.txt      
  inflating: Stocks/eses.us.txt      
  inflating: Stocks/esg.us.txt       
  inflating: Stocks/esgd.us.txt      
  inflating: Stocks/esge.us.txt      
  inflating: Stocks/esgf.us.txt      
  inflating: Stocks/

In [88]:
files = glob.glob("Data/Stocks/*.txt")
allfiles = np.sort(np.array(files))

In [94]:
allfiles[:10]

array(['Data/Stocks/a.us.txt', 'Data/Stocks/aa.us.txt',
       'Data/Stocks/aaap.us.txt', 'Data/Stocks/aaba.us.txt',
       'Data/Stocks/aac.us.txt', 'Data/Stocks/aal.us.txt',
       'Data/Stocks/aamc.us.txt', 'Data/Stocks/aame.us.txt',
       'Data/Stocks/aan.us.txt', 'Data/Stocks/aaoi.us.txt'], dtype='<U28')

In [121]:
data = pd.read_csv(allfiles[0])
data = data[['Date', 'Close']]
for i in allfiles[1:11]:
  df = pd.read_csv(i)
  data = data.merge(df[['Date', 'Close']], on = 'Date')

In [124]:
data.head()

,Date,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close
0,2015-11-11,36.901,17.779,24.50,33.38,22.56,43.524,20.75,4.7960,24.649,16.76,24.252
1,2015-11-12,36.734,17.206,25.00,33.23,22.50,43.044,19.17,4.8951,24.182,16.11,23.652
2,2015-11-13,36.030,17.581,25.26,32.19,22.68,42.603,17.50,4.6672,23.795,16.60,23.110
3,2015-11-16,36.577,18.242,25.02,32.94,22.58,41.994,15.58,4.8455,23.923,16.93,23.130
4,2015-11-17,37.607,18.197,24.62,32.86,22.52,41.473,12.59,4.9150,23.418,16.68,23.268


In [125]:
column_names?

Object `column_names` not found.


In [111]:
pd.merge?

In [114]:
data[['Date', 'Close']]

,Date,Close
0,1999-11-18,29.702
1,1999-11-19,27.257
2,1999-11-22,29.702
3,1999-11-23,27.002
4,1999-11-24,27.717
...,...,...
4516,2017-11-06,68.220
4517,2017-11-07,68.250
4518,2017-11-08,68.110
4519,2017-11-09,67.470
